# MTH330 Term Project

In [ ]:
# Group members: Kelly Chu & Shane Wickramasinghe
print("MTH330 Term Project")

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import cumtrapz
from scipy.interpolate import interp1d
from scipy.optimize import brentq

# --- params (hours as unit) ---
B0 = 100.0
r0 = 0.02          # % per hour background drain
r1 = 0.8           # scales usage -> % per hour drain
A = 0.3            # sinusoidal amplitude
omega = 2*np.pi/24 # daily cycle
phi = 0.0
B = 0.1            # baseline usage

# time vector (0..24 hours)
ts = np.linspace(0, 24, 2401)  # step 0.01h ~ 0.6 min

def u_of_t(t):
    # PURE sinusoidal model
    return A*np.sin(omega*t + phi) + B

# compute usage + drain
u_vals = u_of_t(ts)
drain = r0 + r1 * u_vals

# battery curve (percent)
I = cumtrapz(drain, ts, initial=0.0)   # percent consumed
B_vals = B0 - I

# ---- derivative-based extrema detection ----
du_dt = np.gradient(u_vals, ts)
dDrain_dt = r1 * du_dt

# locate sign changes of derivative
idx = np.where(np.sign(dDrain_dt[:-1]) * np.sign(dDrain_dt[1:]) <= 0)[0]
critical_ts = []
for i in idx:
    a, b = ts[i], ts[i+1]
    try:
        root = brentq(lambda x: r1 * np.interp(x, ts, du_dt), a, b)
        critical_ts.append(root)
    except ValueError:
        pass

# classify minima & maxima using 2nd derivative
d2u_dt2 = np.gradient(du_dt, ts)
mins, maxs = [], []
for t0 in critical_ts:
    val = np.interp(t0, ts, d2u_dt2)
    if val > 0:
        mins.append(t0)
    else:
        maxs.append(t0)

print("Local minima (hours):", np.round(mins,3))
print("Local maxima (hours):", np.round(maxs,3))

# ---- plotting ----
plt.figure(figsize=(10,3))
plt.plot(ts, u_vals, label='u(t)')
plt.scatter(mins, [u_of_t(x) for x in mins], color='green', label='minima')
plt.scatter(maxs, [u_of_t(x) for x in maxs], color='red', label='maxima')
plt.xlabel('Time (hours)')
plt.legend()
plt.title('Usage intensity (sinusoidal only)')
plt.grid(True)
plt.show()

plt.figure(figsize=(10,3))
plt.plot(ts, B_vals, label='Battery %')
plt.xlabel('Time (hours)')
plt.ylabel('Battery %')
plt.title('Battery over 24 hours')
plt.grid(True)
plt.show()
